<a href="https://colab.research.google.com/github/grojasc/Proyecto_aplicado_I_2022/blob/main/Proyecto_Aplicado_I_Demo_GRUPO_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instalar Librerías
---


In [1]:
!pip install streamlit-authenticator
!pip install path

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 593 kB 18.8 MB/s 
     |████████████████████████████████| 9.2 MB 12.9 MB/s 
     |████████████████████████████████| 4.4 MB 48.0 MB/s 
     |████████████████████████████████| 78 kB 3.8 MB/s 
     |████████████████████████████████| 182 kB 57.6 MB/s 
     |████████████████████████████████| 237 kB 31.9 MB/s 
     |████████████████████████████████| 164 kB 56.0 MB/s 
     |████████████████████████████████| 4.7 MB 18.0 MB/s 
     |████████████████████████████████| 62 kB 375 kB/s 
     |████████████████████████████████| 51 kB 2.9 MB/s 
  Created wheel for streamlit-authenticator: filename=streamlit_authenticator-0.2.1-py3-none-any.whl size=10198 sha256=fb19c5c3e12c785e74627e623a8a71263963c63d091af6cb600a011bb4f561b1
  Stored in directory: /root/.cache/pip/wheels/3c/be/0a/a1ee2460cd46bfbc56d9d6d0530828eda02949737cb6968fcf
  Created wheel for validators: file

In [2]:
!pip install streamlit==1.14.1


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.2 MB 19.7 MB/s 
  Attempting uninstall: streamlit
    Found existing installation: streamlit 1.15.2
    Uninstalling streamlit-1.15.2:
      Successfully uninstalled streamlit-1.15.2


In [3]:
import streamlit as st
import pandas as pd
import numpy as np
from PIL import Image
import streamlit.components.v1 as components

## Definición de Funciones


In [4]:
import os 
from PIL import Image, ImageOps
import numpy as np
import pandas as pd


def get_item_image(item_id, resize=True, width=100, height = 150):
    
    path = 'results/images/'+item_id+'.jpeg'
    image = Image.open(path)
    
    if resize:
        basewidth = width
        wpercent = (basewidth / float(image.size[0]))
        hsize = int((float(image.size[1]) * float(wpercent)))
        image = image.resize((width, height), Image.ANTIALIAS)
    image = ImageOps.expand(image, 2)
        
    return image    


def get_rcmnds(customer_data):
    
    combined_rcmnds = customer_data.combined_rcmnds.values
    tfrs_rcmnds = customer_data.tfrs_rcmnds.values
    image_rcmnds = customer_data.image_rcmnds.values
    text_rcmnds = customer_data.text_rcmnds.values
    feature_rcmnds = customer_data.feature_rcmnds.values

    return image_rcmnds, text_rcmnds, feature_rcmnds, tfrs_rcmnds, combined_rcmnds

def get_rcmnds_scores(customer_data):
    
    combined_rcmnds = customer_data.combined_scores.values
    tfrs_rcmnds = customer_data.tfrs_scores.values
    image_rcmnds = customer_data.image_scores.values
    text_rcmnds = customer_data.text_scores.values
    feature_rcmnds = customer_data.feature_scores.values

    return image_rcmnds, text_rcmnds, feature_rcmnds, tfrs_rcmnds, combined_rcmnds

def get_rcmnds_images(image_rcmnds, text_rcmnds, feature_rcmnds, tfrs_rcmnds, combined_rcmnds):
    
    combined_rcmnds = [get_item_image(str(i)) for i in combined_rcmnds]
    tfrs_rcmnds = [get_item_image(str(i)) for i in tfrs_rcmnds]
    image_rcmnds = [get_item_image(str(i)) for i in image_rcmnds]
    text_rcmnds = [get_item_image(str(i)) for i in text_rcmnds]
    feature_rcmnds = [get_item_image(str(i)) for i in feature_rcmnds]
    
    return image_rcmnds, text_rcmnds, feature_rcmnds, tfrs_rcmnds, combined_rcmnds


def get_rcmnds_features(df, image_rcmnds, text_rcmnds, feature_rcmnds, tfrs_rcmnds, combined_rcmnds):
    
    combined_rcmnds = df[df.article_id.isin(combined_rcmnds)][['product_type_name', 'colour_group_name', 'department_name']]
    tfrs_rcmnds = df[df.article_id.isin(tfrs_rcmnds)][['product_type_name', 'colour_group_name', 'department_name']]
    image_rcmnds = df[df.article_id.isin(image_rcmnds)][['product_type_name', 'colour_group_name', 'department_name']]
    text_rcmnds = df[df.article_id.isin(text_rcmnds)][['product_type_name', 'colour_group_name', 'department_name']]
    feature_rcmnds = df[df.article_id.isin(feature_rcmnds)][['product_type_name', 'colour_group_name', 'department_name']]
    
    return image_rcmnds, text_rcmnds, feature_rcmnds, tfrs_rcmnds, combined_rcmnds
    

def get_rcmnds_desc(df, image_rcmnds, text_rcmnds, feature_rcmnds, tfrs_rcmnds, combined_rcmnds):
    
    combined_rcmnds = df[df.article_id.isin(combined_rcmnds)].detail_desc
    tfrs_rcmnds = df[df.article_id.isin(tfrs_rcmnds)].detail_desc
    image_rcmnds = df[df.article_id.isin(image_rcmnds)].detail_desc
    text_rcmnds = df[df.article_id.isin(text_rcmnds)].detail_desc
    feature_rcmnds = df[df.article_id.isin(feature_rcmnds)].detail_desc
    
    return image_rcmnds, text_rcmnds, feature_rcmnds, tfrs_rcmnds, combined_rcmnds

## Crear App con Streamlit
---

In [ ]:
from PIL import Image

In [15]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import pickle
import streamlit_authenticator as sauth
from PIL import Image

def main():
  st.title("Alamacenes La Pelar")
  #st.subheader("Siempre todo original")
  img = Image.open("/content/descarga.png")
  st.image(img,width=100,caption="La Pelar")
  st.markdown("""### _Elige los mejores productos originales de nuestras marcas adides y under amor_ :sunglasses:""",True)

  menu = ["Home","About","Login","SignUp"]
  choice = st.sidebar.selectbox('Menu',menu)
  if choice == 'Home':
    st.subheader("Inicio")	

  elif choice == "Login":
    st.subheader("Login Section")	
    username  = st.sidebar.text_input("User Name")
    password = st.sidebar.text_input("Password",type='password' )
    if st.sidebar.checkbox("Login"):
      if password == '12345':
        st.success("Logged In as {}".format(username))

        file = st.file_uploader("Subir aquí la imagen ", type=["jpg", "jpeg", "png"])
        if file is None:
          st.text("Por favor subir una imagen")
      else:
        st.warning("Incorrect UserName/Password")

  elif choice == "SignUp":
    st.subheader("Create New Account")	
    new_user = st.text_input("UserName")
    new_password = st.text_input("Password",type = 'password')
    if st.button("SignUp"):
      st.success("you have successfully created an Account")
      st.info("Go to Login Menu")


















if __name__ == '__main__':
  main()

Overwriting app.py


In [16]:
!streamlit run app.py & npx localtunnel --port 8501

npx: installed 22 in 2.556s



  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.221.50.128:8501

your url is: https://wild-ties-hang-35-221-50-128.loca.lt
  Stopping...
^C


In [13]:
!pip install pyngrok
from pyngrok import ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
!nohub streamlit run app.py &

/bin/bash: nohub: command not found


In [10]:
!streamlit run app.py &>/dev/null&

In [7]:
!ngrok authtoken 2HyQaq011oRwNjSbdm1xI1TWAXh_5CwrTGKbM9avvivrkT4m7

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [11]:
# Setup a tunnel to the streamlit port 8501
public_url = ngrok.connect(port='8501')
public_url

INFO:pyngrok.ngrok:Opening tunnel named: http-80-f951d949-7bb9-4697-a9c4-7ebd3249888a
2022-12-06 00:28:21.467 INFO    pyngrok.ngrok: Opening tunnel named: http-80-f951d949-7bb9-4697-a9c4-7ebd3249888a
INFO:pyngrok.process.ngrok:t=2022-12-06T00:28:21+0000 lvl=info msg="no configuration paths supplied"
2022-12-06 00:28:21.505 INFO    pyngrok.process.ngrok: t=2022-12-06T00:28:21+0000 lvl=info msg="no configuration paths supplied"
INFO:pyngrok.process.ngrok:t=2022-12-06T00:28:21+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml
2022-12-06 00:28:21.519 INFO    pyngrok.process.ngrok: t=2022-12-06T00:28:21+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml
INFO:pyngrok.process.ngrok:t=2022-12-06T00:28:21+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil
2022-12-06 00:28:21.526 INFO    pyngrok.process.ngrok: t=2022-12-06T00:28:21+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngro

<NgrokTunnel: "http://1b03-35-221-50-128.ngrok.io" -> "http://localhost:80">

INFO:pyngrok.process.ngrok:t=2022-12-06T00:28:21+0000 lvl=info msg="started tunnel" obj=tunnels name=http-80-f951d949-7bb9-4697-a9c4-7ebd3249888a addr=http://localhost:80 url=https://1b03-35-221-50-128.ngrok.io
2022-12-06 00:28:21.696 INFO    pyngrok.process.ngrok: t=2022-12-06T00:28:21+0000 lvl=info msg="started tunnel" obj=tunnels name=http-80-f951d949-7bb9-4697-a9c4-7ebd3249888a addr=http://localhost:80 url=https://1b03-35-221-50-128.ngrok.io
INFO:pyngrok.process.ngrok:t=2022-12-06T00:28:21+0000 lvl=info msg=end pg=/api/tunnels id=77d1f0ea34baa821 status=201 dur=51.537838ms
2022-12-06 00:28:21.712 INFO    pyngrok.process.ngrok: t=2022-12-06T00:28:21+0000 lvl=info msg=end pg=/api/tunnels id=77d1f0ea34baa821 status=201 dur=51.537838ms
INFO:pyngrok.process.ngrok:t=2022-12-06T00:28:21+0000 lvl=info msg=start pg="/api/tunnels/http-80-f951d949-7bb9-4697-a9c4-7ebd3249888a (http)" id=913b7df41349c4f6
2022-12-06 00:28:21.721 INFO    pyngrok.process.ngrok: t=2022-12-06T00:28:21+0000 lvl=info 

In [12]:
!pgrep streamlit

330


In [ ]:
kill 445